# HFSS to SBR+ time animation

This example shows how to use PyAEDT to create an SBR+ time animation
and save it to a GIF file. This example works only on CPython.

Keywords: **HFSS**, **SBR+**, **time domain**, **IFFT**.

## Perform imports and define constants

Perform required imports.

In [1]:
import os
import tempfile
import time

from ansys.aedt.core import Hfss
from ansys.aedt.core.examples.downloads import download_sbr_time

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Download project

In [4]:
project_file = download_sbr_time(local_path=temp_folder.name)

## Launch HFSS and analyze

In [5]:
hfss = Hfss(
    project=project_file,
    version=AEDT_VERSION,
    non_graphical=NG_MODE,
    new_desktop=True,
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpn08lf39n.ansys\sbr\poc_scat_small.aedt.


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


...

PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project poc_scat_small has been opened.


PyAEDT INFO: Active Design set to HFSSDesign1


PyAEDT INFO: Active Design set to HFSSDesign1


PyAEDT INFO: Aedt Objects correctly read


In [6]:
hfss.analyze(cores=NUM_CORES)

PyAEDT INFO: Project poc_scat_small Saved correctly


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/HFSS correctly changed.


PyAEDT INFO: Solving all design setups. Analysis started...


PyAEDT INFO: Design setup None solved correctly in 0.0h 1.0m 14.0s


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/HFSS correctly changed.


True

## Get solution data

Get solution data. After the simulation is performed, you can load solutions
in the ``solution_data`` object.

In [7]:
solution_data = hfss.post.get_solution_data(
    expressions=["NearEX", "NearEY", "NearEZ"],
    variations={"_u": ["All"], "_v": ["All"], "Freq": ["All"]},
    context="Near_Field",
    report_category="Near Fields",
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpn08lf39n.ansys\sbr\poc_scat_small.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpn08lf39n.ansys\sbr\poc_scat_small.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.0802009105682373


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Solution Data Correctly Loaded.


Time to initialize solution data:0.0220947265625
Time to initialize solution data:0.11086797714233398


## Compute IFFT

Compute IFFT (Inverse Fast Fourier Transform).

In [8]:
t_matrix = solution_data.ifft("NearE", window=True)

## Export IFFT to CSV file

Export IFFT to a CSV file.

In [9]:
frames_list_file = solution_data.ifft_to_file(
    coord_system_center=[-0.15, 0, 0],
    db_val=True,
    csv_path=os.path.join(hfss.working_directory, "csv"),
)

## Plot scene

Plot the scene to create the time plot animation

In [10]:
hfss.post.plot_scene(
    frames=frames_list_file,
    gif_path=os.path.join(hfss.working_directory, "animation.gif"),
    norm_index=15,
    dy_rng=35,
    show=False,
    view="xy",
    zoom=1,
)

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


Widget(value='<iframe src="http://localhost:59003/index.html?ui=P_0x1c3f4a363b0_0&reconnect=auto" class="pyvis…

## Release AEDT

Release AEDT and close the example.

In [11]:
hfss.save_project()
hfss.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project poc_scat_small Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``. If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell removes all temporary files, including the project folder.

In [12]:
temp_folder.cleanup()